<a href="https://colab.research.google.com/github/MAYANKTIGER/Image_Clssification_CNN/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.0 MB/s 


In [ ]:
"""KerasTuner 
KerasTuner is an easy-to-use, scalable hyperparameter optimization framework that solves the pain points of
 hyperparameter search. Easily configure your search space with a define-by-run syntax, then leverage one of the
  available search algorithms to find the best hyperparameter values for your models. KerasTuner comes with Bayesian Optimization,
   Hyperband, and Random Search algorithms built-in, and is also designed to be easy for researchers to extend in order to experiment
    with new search algorithms.""" 

In [3]:
# Import KerasTuner and TensorFlow 
import tensorflow as tf
from tensorflow import keras
import numpy as np 

In [5]:
fashion_mnist = keras.datasets.fashion_mnist 
# keras.datasets will provide the data
# fasion_mnist data consist of 60000(train_data)and 10000(test_data) images and Each image has a 28x28 grayscale pixels 
# Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value 
# associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker.
# This pixel-value is an integer between 0 and 255. The training and test data sets have 785 columns

In [7]:
(train_images, train_lables), (test_images, test_lables) = fashion_mnist.load_data()
  # Load MNIST data into train and test sets

4431872/4422102 [==============================] - 0s 0us/step


In [8]:
train_images[0][4]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         0,  36, 136, 127,  62,  54,   0,   0,   0,   1,   3,   4,   0,
         0,   3], dtype=uint8)

In [10]:
train_images = train_images/250.0
test_images =test_images/250.0
# normalize the pixel value and equally divided by the 250  
# scaled down 0-1
# it will help to quickly training the Model 

In [11]:
train_images[0][4]

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.012, 0.   , 0.144, 0.544, 0.508, 0.248,
       0.216, 0.   , 0.   , 0.   , 0.004, 0.012, 0.016, 0.   , 0.   ,
       0.012])

In [20]:
# Now will reshape the data 
train_images.shape

(60000, 28, 28, 1)

In [14]:
train_images = train_images.reshape(len(train_images), 28,28,1)
test_images = test_images.reshape(len(test_images), 28,28,1)
#  4D array as input to the CNN 
#  Keras expects image dimension as (Width, Height, Channels)

In [16]:
train_images.shape

(60000, 28, 28, 1)

In [51]:
#train_images[0][4]

In [18]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [19]:
from keras_tuner import RandomSearch
# from keras_tuner.engine.hyperparameters import HyperParameters

In [22]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=3,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_lables,epochs=3,validation_split=0.1)

In [23]:
tuner_search.search(train_images,train_lables,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 13m 23s]
val_accuracy: 0.8684999942779541

Best val_accuracy So Far: 0.8684999942779541
Total elapsed time: 01h 03m 08s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [24]:
model=tuner_search.get_best_models(num_models=1)[0]

In [25]:
model.fit(train_images, train_lables, epochs=6, validation_split=0.1, initial_epoch=3)

Epoch 4/6
1688/1688 [==============================] - 259s 153ms/step - loss: 0.2892 - accuracy: 0.8920 - val_loss: 0.3830 - val_accuracy: 0.8722
Epoch 5/6
1688/1688 [==============================] - 260s 154ms/step - loss: 0.2654 - accuracy: 0.9009 - val_loss: 0.3742 - val_accuracy: 0.8755
Epoch 6/6
1688/1688 [==============================] - 261s 154ms/step - loss: 0.2501 - accuracy: 0.9075 - val_loss: 0.4180 - val_accuracy: 0.8735


In [17]:
# labels_data =  np.unique(test_lables)
# for i in labels_data:

In [26]:
new_input = test_images[0].reshape(1,28,28,1)

In [29]:
test_lables[0] # Ankle boot

9

In [31]:
predictions = model.predict(new_input)
print(np.argmax(predictions[0]))

9


In [33]:
predictions_1 = model.predict(new_input)
print(predictions_1)

[[2.3411252e-15 1.1437234e-22 6.2588891e-24 1.8492557e-17 2.5504930e-23
  2.4522715e-11 3.4252701e-26 7.5349016e-09 7.2496906e-19 1.0000000e+00]]


In [34]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [40]:
print("This image is similar to the {}".format(class_names[np.argmax(predictions)]))

This image is similar to the Ankle boot
